using model 5
final 0.87

In [ ]:
import sys, skvideo.io, json, base64
import numpy as np
from PIL import Image
from io import BytesIO, StringIO
import tensorflow as tf
file = sys.argv[-1]

load_model_path = 'work/model/model'

if file == 'demo.py':
  print ("Error loading video")
  quit
# Define softmax function
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return (np.exp(x.T) / np.sum(np.exp(x.T), axis=0)).T

# Define encoder function
def encode(array):
	pil_img = Image.fromarray(array)
	buff = BytesIO()
	pil_img.save(buff, format="PNG")
	return base64.b64encode(buff.getvalue()).decode("utf-8")

video = skvideo.io.vread(file)

# Frame numbering starts at 1
tf.reset_default_graph()
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph(load_model_path+'.meta')
    new_saver.restore(sess, load_model_path)
    graph = tf.get_default_graph()
    input_image = graph.get_tensor_by_name('input_image:0')
    nn_last_layer = graph.get_tensor_by_name('nn_last_layer:0')
    answer_key = {}
    frame = 1
    crop_up = 0
    crop_down = 512
    image_shape = [600,800,3]
    for rgb_frame in video:
        crop_frame = rgb_frame[crop_up:crop_down,:,:]
        crop_frame = np.expand_dims(crop_frame,0)
        out = sess.run(nn_last_layer, feed_dict={input_image: crop_frame})[0]
        out = softmax(out)
        # Look for red cars :)
        binary_car_result = np.where(out[:,:,1]>0.2,1,0)
#         binary_car_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_car_result), axis=0)
        binary_car_result = np.concatenate((binary_car_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        # Look for road :)
        binary_road_result = np.where(out[:,:,2]>0.6,1,0)
#         binary_road_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_road_result), axis=0)
        binary_road_result = np.concatenate((binary_road_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        answer_key[frame] = [encode(binary_car_result.astype('uint8')), encode(binary_road_result.astype('uint8'))]
        # Increment frame
        frame+=1


# Print output in proper json format
print (json.dumps(answer_key))
